In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from loess.loess_1d import loess_1d

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas.coverages import CoverageInternal

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
coverage_identifier = "tas_seasonal_absolute_model_ensemble-rcp26-DJF"
point_coords = "POINT(11.5469 44.9524)"
date_range = "../.."

cov = CoverageInternal(
    configuration=db.get_coverage_configuration_by_coverage_identifier(session, coverage_identifier), 
    identifier=coverage_identifier
)

In [3]:
settings.debug = False

In [ ]:
new_time_start = time.time()
coverage_series, observation_series = operations.get_coverage_time_series(
    settings=settings,
    session=session,
    coverage=cov,
    point_geom=shapely.io.from_wkt(point_coords),
    temporal_range=date_range,
    coverage_smoothing_strategies=[
        CoverageDataSmoothingStrategy.NO_SMOOTHING,
        CoverageDataSmoothingStrategy.MOVING_AVERAGE_11_YEARS,
        CoverageDataSmoothingStrategy.LOESS_SMOOTHING
    ],
    observation_smoothing_strategies=[
        ObservationDataSmoothingStrategy.NO_SMOOTHING,
        ObservationDataSmoothingStrategy.MOVING_AVERAGE_5_YEARS
    ],
    include_coverage_data=True,
    include_observation_data=False,
    include_coverage_uncertainty=True,
    include_coverage_related_data=True
)
new_time_end = time.time()
print(f"elapsed: {new_time_end - new_time_start}")

In [9]:
[(c.identifier, s.value) for c, s in coverage_series.keys()]

[('tas_seasonal_absolute_model_mpi_esm_lr_remo2009-rcp26-DJF', 'NO_SMOOTHING'),
 ('tas_seasonal_absolute_model_mpi_esm_lr_remo2009-rcp26-DJF',
  'MOVING_AVERAGE_11_YEARS'),
 ('tas_seasonal_absolute_model_mpi_esm_lr_remo2009-rcp26-DJF',
  'LOESS_SMOOTHING'),
 ('tas_seasonal_absolute_model_ec_earth_racmo22e-rcp26-DJF', 'NO_SMOOTHING'),
 ('tas_seasonal_absolute_model_ec_earth_racmo22e-rcp26-DJF',
  'MOVING_AVERAGE_11_YEARS'),
 ('tas_seasonal_absolute_model_ec_earth_racmo22e-rcp26-DJF',
  'LOESS_SMOOTHING'),
 ('tas_seasonal_absolute_model_ensemble-rcp26-DJF', 'NO_SMOOTHING'),
 ('tas_seasonal_absolute_model_ensemble-rcp26-DJF', 'MOVING_AVERAGE_11_YEARS'),
 ('tas_seasonal_absolute_model_ensemble-rcp26-DJF', 'LOESS_SMOOTHING'),
 ('tas_seasonal_absolute_model_ec_earth_cclm4_8_17-rcp26-DJF', 'NO_SMOOTHING'),
 ('tas_seasonal_absolute_model_ec_earth_cclm4_8_17-rcp26-DJF',
  'MOVING_AVERAGE_11_YEARS'),
 ('tas_seasonal_absolute_model_ec_earth_cclm4_8_17-rcp26-DJF',
  'LOESS_SMOOTHING'),
 ('tas_seas

In [12]:
next(iter(coverage_series.values()))

time
1976-02-15 12:00:00+00:00           2.954980
1977-02-14 17:54:11.613000+00:00    3.255182
1978-02-14 23:48:23.226000+00:00    2.340784
1979-02-15 05:42:34.839000+00:00    4.659448
1980-02-15 11:36:46.452000+00:00    3.844202
                                      ...   
2096-02-15 00:23:13.548000+00:00    4.219568
2097-02-14 06:17:25.161000+00:00    5.275751
2098-02-14 12:11:36.774000+00:00    4.540552
2099-02-14 18:05:48.387000+00:00    4.133020
2100-02-15 00:00:00+00:00           4.696100
Name: tas_seasonal_absolute_model_mpi_esm_lr_remo2009-rcp26-DJF, Length: 125, dtype: float64

In [3]:
cov.configuration.retrieve_configuration_parameters(coverage_identifier)

{'scenario': 'rcp26'}

In [8]:
{pv.configuration_parameter_value.configuration_parameter.name: pv.configuration_parameter_value.name for pv in cov.configuration.retrieve_used_values(coverage_identifier)}

{'scenario': 'rcp26'}

In [12]:
{
    pv.configuration_parameter_value.configuration_parameter.name: pv.configuration_parameter_value.configuration_parameter.display_name_english 
    for pv in cov.configuration.retrieve_used_values(coverage_identifier)
}

{'scenario': 'Scenario'}